In [1]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
from dataset import CircularLearningRate

from tensorflow import config
assert config.list_physical_devices('GPU')

In [2]:
BATCH_SIZE = 32
MAXLEN = 64

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)
KINDS_SIZE = len(dataset.KINDS)

def build_model(EMBED_DIM=210, UNITS=420):

    layer = input_text = keras.Input(batch_shape=(None, MAXLEN), batch_size=BATCH_SIZE)
    
    layer = layers.Embedding(LETTERS_SIZE, EMBED_DIM, input_length=MAXLEN, mask_zero=True)(layer)
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True), merge_mode='sum')(layer)
    layer = layers.add([layer, 
            layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True), merge_mode='sum')(layer)])
    
    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    model = keras.Model(inputs=[input_text], outputs=outputs)

    jsmodel = model 
    # keras.utils.plot_model(model, to_file='model.png')
    return model, jsmodel

model, jsmodel = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 64, 210)      9240        input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 64, 420)      2120160     embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 64, 420)      2825760     bidirectional[0][0]              
______________________________________________________________________________________________

In [3]:
def fit(train_validation, scheduler, verbose=1):
    train, valid = train_validation
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    callbacks = []
    if isinstance(scheduler, CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)
        
    x  = train.normalized
    vx = valid.normalized
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin, 'C': train.normalized } # , 'K': train.kind }
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin, 'C': valid.normalized } # , 'K': valid.kind }
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=1, verbose=verbose, callbacks=callbacks)


In [4]:
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid

In [12]:
data_other = load_data(['biblical', 'garbage', 'poetry'])

In [6]:
data_rabanit = load_data(['rabanit'])

In [7]:
data_pre_modern = load_data(['pre_modern'])

In [8]:
data_modern = load_data(validation=0.2, source=['modern'])

In [13]:
model.load_weights('./checkpoints/uninit')
history = fit(data_other, scheduler=CircularLearningRate(20e-4, 50e-4, 5e-4))
model.save_weights('./checkpoints/other')

Train on 44787 samples, validate on 4976 samples
44787/44787 [==============================] - 89s 2ms/sample - loss: 0.4278 - N_loss: 0.3384 - D_loss: 0.0771 - S_loss: 0.0123 - N_accuracy: 0.8819 - D_accuracy: 0.9692 - S_accuracy: 0.9967 - val_loss: 0.2975 - val_N_loss: 0.2305 - val_D_loss: 0.0622 - val_S_loss: 0.0048 - val_N_accuracy: 0.9225 - val_D_accuracy: 0.9746 - val_S_accuracy: 0.9987


In [14]:
model.load_weights('./checkpoints/other')
history = fit(data_rabanit, scheduler=CircularLearningRate(20e-4, 50e-4, 5e-4))
model.save_weights('./checkpoints/rabanit')

Train on 80969 samples, validate on 1032 samples
80969/80969 [==============================] - 147s 2ms/sample - loss: 0.1252 - N_loss: 0.0940 - D_loss: 0.0285 - S_loss: 0.0027 - N_accuracy: 0.9680 - D_accuracy: 0.9895 - S_accuracy: 0.9994 - val_loss: 0.2556 - val_N_loss: 0.1955 - val_D_loss: 0.0586 - val_S_loss: 0.0028 - val_N_accuracy: 0.9407 - val_D_accuracy: 0.9814 - val_S_accuracy: 0.9993


In [15]:
model.load_weights('./checkpoints/rabanit')
history = fit(data_pre_modern, scheduler=CircularLearningRate(20e-4, 40e-4, 0.1e-4))
model.save_weights('./checkpoints/pre_modern')

Train on 65746 samples, validate on 6819 samples
65746/65746 [==============================] - 127s 2ms/sample - loss: 0.2194 - N_loss: 0.1631 - D_loss: 0.0511 - S_loss: 0.0052 - N_accuracy: 0.9449 - D_accuracy: 0.9792 - S_accuracy: 0.9986 - val_loss: 0.2240 - val_N_loss: 0.1695 - val_D_loss: 0.0472 - val_S_loss: 0.0072 - val_N_accuracy: 0.9439 - val_D_accuracy: 0.9814 - val_S_accuracy: 0.9983


In [16]:
model.load_weights('./checkpoints/pre_modern')
history = fit(data_modern, scheduler=CircularLearningRate(6e-3, 6e-3, 0.1e-3))
# history = fit(data_modern, scheduler=CircularLearningRate(1e-3, 7e-3, 0.5e-3))
model.save_weights('./checkpoints/modern')

Train on 13755 samples, validate on 3924 samples
13755/13755 [==============================] - 37s 3ms/sample - loss: 0.1623 - N_loss: 0.1180 - D_loss: 0.0416 - S_loss: 0.0027 - N_accuracy: 0.9607 - D_accuracy: 0.9844 - S_accuracy: 0.9993 - val_loss: 0.1683 - val_N_loss: 0.1186 - val_D_loss: 0.0472 - val_S_loss: 0.0024 - val_N_accuracy: 0.9606 - val_D_accuracy: 0.9816 - val_S_accuracy: 0.9995


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [ ]:
jsmodel.summary()

In [ ]:
import tensorflowjs as tfjs
jsmodel.load_weights('./checkpoints/modern')
tfjs.converters.save_keras_model(jsmodel, '.')

In [17]:
model.load_weights('./checkpoints/modern')

def print_predictions(data, k):
    s = slice(k*BATCH_SIZE, (k+1)*BATCH_SIZE)
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud[s], data.dagesh[s], data.sin[s]]
    actual = dataset.merge(batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)

print_predictions(data_modern[1], 6)

מצוי:  הַטַּנְקִים, לְאַתֵּר אֶת הַמַּעֲרָכִים הַיִּשְׂרְאֵלִיִּים בְּאוֹפֶן מְדוּיָּיק וְלִפְגּוֹעַ בָּהֶם. כֵּן, 
רצוי:  הַטַּנְקִים, לְאַתֵּר אֶת הַמַּעֲרָכִים הַיִּשְׂרְאֵלִיִּים בְּאוֹפֶן מְדוּיָּיק וְלִפְגּוֹעַ בָּהֶם. כֵּן, 
מצוי:  לֹא נִלְקְחוּ בְּחֶשְׁבּוֹן עַרְפִילֵי הַבּוֹקֶר הַכְּבֵדִים הַמְּכַסִּים אֶת הַקַּרְקַע הַנְּמוּכָה 
רצוי:  לֹא נִלְקְחוּ בְּחֶשְׁבּוֹן עַרְפִילֵּי הַבּוֹקֶר הַכְּבֵדִים הַמְּכַסִּים אֶת הַקַּרְקַע הַנְּמוּכָה 
מצוי:  מִמִּזְרָח לְתַעֲלֶה בִּתְקוּפָה זוֹ שֶׁל הַשָּׁנָה, אֲשֶׁר הִקְשׁוּ עַל שְׁמִירַת כִּיוּוּן הַתְּנוּעָה 
רצוי:  מִמִּזְרָח לַתְּעָלָה בִּתְקוּפָה זוֹ שֶׁל הַשָּׁנָה, אֲשֶׁר הִקְשׁוּ עַל שְׁמִירַת כִּיוּוּן הַתְּנוּעָה 
מצוי:  עַל יְדֵי הַכּוֹחוֹת. כָּל זֶה הִשְׁפִּיעַ עַל בִּיצּוּעֵי הַכּוֹחוֹת הַמִּצְרִיִּים שֶׁנֶּעֶצְרוּ 
רצוי:  עַל יְדֵי הַכּוֹחוֹת. כָּל זֶה הִשְׁפִּיעַ עַל בִּיצּוּעֵי הַכּוֹחוֹת הַמִּצְרִיִּים שֶׁנֶּעֶצְרוּ 
מצוי:  בָּרֶגַע שֶׁנִּתְקְלוּ בְּהִתְנַגְּדוּת מִצַּד הַצָּבָא הַיִּשְׂרְאֵלִי, וְלֹא גִּילוּ הִתְאַמְּצוּ ל

In [ ]:
shutil.rmtree(os.sep.join([tempfile.gettempdir(), '.tensorboard-info']), ignore_errors=True)
shutil.rmtree('logs', ignore_errors=True)
os.makedirs('logs')
# %tensorboard --logdir logs

In [ ]:
print(data_modern[1].text[0])
print(data_modern[1].text[1])